In [1]:
import pandas as pd


input_file = 'data/523_paid_subscription/startup_users_visits.csv'
df = pd.read_csv(input_file)

df['date'] = pd.to_datetime(df['date'])
first_visits = df.groupby('user_id')['date'].min().reset_index()
first_visits.rename(columns={'date': 'first_date'}, inplace=True)
first_visits['window_end'] = first_visits['first_date'] + pd.Timedelta(days=90)
first_visits['first_month'] = first_visits['first_date'].values.astype('datetime64[M]')
last_date = df['date'].max()
months_to_include = first_visits.groupby('first_month')['window_end'].max().le(last_date).reset_index()
months_to_include = months_to_include[months_to_include['window_end']].copy()
qualified_users = first_visits[first_visits['first_month'].isin(months_to_include['first_month'])]
df_qualified = df[df['user_id'].isin(qualified_users['user_id'])]
df_qualified = df_qualified.merge(qualified_users[['user_id', 'first_date', 'window_end', 'first_month']], on='user_id', how='left')

payments_within_window = df_qualified[
    (df_qualified['pay'] == True) &
    (df_qualified['date'] >= df_qualified['first_date']) &
    (df_qualified['date'] <= df_qualified['window_end'])
].drop_duplicates('user_id')

payment_counts = payments_within_window.groupby('first_month').size().reset_index(name='count')
payment_summary = payment_counts.sort_values('first_month')
payment_summary['first_day_of_month'] = payment_summary['first_month'].dt.strftime('%Y-%m-01')
final_output = payment_summary[['first_day_of_month', 'count']]
print(final_output.to_csv(index=False, header=False))

2022-11-01,133
2022-12-01,552
2023-01-01,437
2023-02-01,949
2023-03-01,950
2023-04-01,678
2023-05-01,954
2023-06-01,848

